# Command

> Permite desconectar o autor de uma solicitação do objeto que deverá executar a ação solicitada. Encapsula uma solicitação como um objeto, o que lhe permite parametrizar outros objetos com diferentes solicitações.


```bash
Screen
|_cut()
|_past()
|_clear_clipboard()
|_length()

ScreenCommand
    |_execute()
    |_undo()
    
  ^
  |_
    |_CutCommand(ScreenCommand)
    |_PasteCommand(ScreenCommand)
    

ScreenInvoker
|_store_and_execute(self, command)
|    |_command.execute()
|_undo_last()

```

In [1]:
class Screen(object):
    
    def __init__(self, text=''):
        self.text = text
        self.clip_board = ''
        
    def cut(self, start=0, end=0):
        self.clip_board = self.text[start:end]
        self.text = self.text[:start] + self.text[end:]
        
    def past(self, offset=0):
        self.text = self.text[:offset] + self.clip_board + self.text[offset:]
        
    def clear_clipboard(self):
        self.clip_board = ''
        
    def length(self):
        return len(self.text)
    
    def __str__(self):
        return self.text

In [2]:
# interface do Command

class ScreenCommand:
    def __init__(self, screen):
        self.screen = screen
        self.previous_state = screen.text
        
    def execute(self):
        pass
    
    def undo(self):
        pass

In [3]:
class CutCommand(ScreenCommand):
    def __init__(self, screen, start=0, end=0):
        super().__init__(screen)
        self.start = start
        self.end = end
        
    def execute(self):
        self.screen.cut(start=self.start, end=self.end)
        
    def undo(self):
        self.screen.clear_clipboard()
        self.screen.text = self.previous_state

In [4]:
class PasteCommand(ScreenCommand):
    def __init__(self, screen, offset=0):
        super().__init__(screen)
        self.offset = offset
        
    def execute(self):
        self.screen.past(offset=self.offset)
        
    def undo(self):
        self.screen.clear_clipboard()
        self.screen.text = self.previous_state

In [5]:
class ScreenInvoker:
    def __init__(self):
        self.history = []
    
    def store_and_execute(self, command):
        command.execute()
        self.history.append(command)
        
    def undo_last(self):
        if self.history:
            self.history.pop().undo()

In [6]:
screen = Screen('hello world')
screen.__str__()

'hello world'

In [7]:
cut = CutCommand(screen, start=5, end=11)

In [8]:
client = ScreenInvoker()

In [9]:
client.store_and_execute(cut)
screen.__str__()

'hello'

In [10]:
paste = PasteCommand(screen, offset=0)

In [11]:
client.store_and_execute(paste)
screen.__str__()

' worldhello'

In [12]:
client.undo_last()
screen.__str__()

'hello'

In [13]:
client.undo_last()
screen.__str__()

'hello world'